<a href="https://colab.research.google.com/github/AUMANSH/Data-Science-Projects/blob/main/federated_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Federated Learning Using CIFAR-10 Dataset</b></div>

![_f0b10fc2-0595-441b-8c75-e554671b3088.jpg](attachment:d8d59f78-0929-4986-a7af-187a3bba0382.jpg)

Federated Learning (FL) is used to train machine learning models across decentralized devices or data sources without sharing raw data. It is valuable because:

Privacy and Security: Keeps data local, preserving user privacy and meeting regulations like GDPR.

Decentralization: Enables training across multiple devices or organizations without centralizing data.

Efficiency: Reduces data transfer by sharing only model updates, saving bandwidth and resources.

Personalization: Creates models tailored to local data while contributing to a global model.

Scalability: Scales to thousands of clients, ideal for IoT, edge devices, and large networks.

Collaboration: Allows organizations to train models together without sharing sensitive data.

FL is used in healthcare, mobile apps, finance, and IoT, offering privacy-preserving, efficient, and collaborative machine learning solutions.

<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b>Install Necessary Library </b></div>

In [ ]:
!pip install flwr

<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Import Libraries and Load CIFAR-10 Dataset</b></div>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

import warnings
warnings.filterwarnings('ignore')


# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize images to the range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print(f"Train shape: {x_train.shape}, {y_train.shape}")
print(f"Test shape: {x_test.shape}, {y_test.shape}")

Train shape: (50000, 32, 32, 3), (50000, 10)
Test shape: (10000, 32, 32, 3), (10000, 10)


<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Split Data Across Clients</b></div>

Simulate data distribution for federated learning by splitting the dataset among clients.

In [ ]:
def split_data(x, y, num_clients):
    client_data = []
    shard_size = len(x) // num_clients
    for i in range(num_clients):
        start = i * shard_size
        end = start + shard_size
        client_data.append((x[start:end], y[start:end]))
    return client_data

# Split data for 3 clients
num_clients = 3
client_data = split_data(x_train, y_train, num_clients)

<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Define the Model</b></div>

In [ ]:
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Initialize the Global Model </b></div>

In [ ]:
# Initialize the global model
global_model = create_model()

# Each local model starts with the same initial weights
initial_weights = global_model.get_weights()


<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Federated Learning Training</b></div>

In [ ]:
# Number of rounds for federated learning
num_rounds = 10
for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}")

    local_weights = []

    # Train on each client
    for client_id in range(num_clients):
        print(f"Training on client {client_id + 1}")

        # Create local model and set global weights
        local_model = create_model()
        local_model.set_weights(global_model.get_weights())

        # Get client data
        X, y = client_data[client_id]

        # Train local model
        local_model.fit(X, y, epochs=1, batch_size=32, verbose=0)

        # Collect local model weights
        local_weights.append(local_model.get_weights())

    # Federated averaging: Aggregate local weights
    averaged_weights = [np.mean([local_weights[j][i] for j in range(num_clients)], axis=0)
                        for i in range(len(local_weights[0]))]

    # Update global model weights
    global_model.set_weights(averaged_weights)

    # Evaluate the global model
    loss, accuracy = global_model.evaluate(x_test, y_test, verbose=0)
    print(f"Global model accuracy after round {round_num + 1}: {accuracy:.4f}")


Round 1/10
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 1: 0.4043
Round 2/10
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 2: 0.5751
Round 3/10
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 3: 0.6129
Round 4/10
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 4: 0.6457
Round 5/10
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 5: 0.6574
Round 6/10
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 6: 0.6755
Round 7/10
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 7: 0.6855
Round 8/10
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 8: 0.6932
Round 9/10
Training on client 1
Training on client 2
Tra

<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Define Flower Client</b></div>

Flower clients handle local training and evaluation. Each client will train its model using its local data.

In [ ]:
import flwr as fl

class CIFARClient(fl.client.NumPyClient):
    def __init__(self, model, train_data, test_data):
        self.model = model
        self.train_data = train_data
        self.test_data = test_data

    def get_parameters(self):
        # Return model parameters
        return self.model.get_weights()

    def set_parameters(self, parameters):
        # Set model parameters
        self.model.set_weights(parameters)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        x_train, y_train = self.train_data
        self.model.fit(x_train, y_train, epochs=1, batch_size=32, verbose=0)
        return self.get_parameters(), len(x_train), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        x_test, y_test = self.test_data
        loss, accuracy = self.model.evaluate(x_test, y_test, verbose=0)
        return float(loss), len(x_test), {"accuracy": float(accuracy)}


<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Start the Flower Server</b></div>

In [ ]:
def start_flower_server():
    fl.server.start_server(
        server_address="0.0.0.0:8080",
        config=fl.server.ServerConfig(num_rounds=3),
    )


<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Start the Flower Client</b></div>

In [ ]:
def start_flower_client(client_id):
    # Get client-specific data
    train_data = client_data[client_id]
    test_data = (x_test, y_test)

    # Create model
    model = create_model()

    # Create and start the client
    client = CIFARClient(model, train_data, test_data)
    fl.client.start_numpy_client(server_address="0.0.0.0:8080", client=client)

<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Run Federated Learning</b></div>

Run the server and clients in parallel. For local testing, you can use threads or separate processes.

Run Server

In [ ]:
import flwr as fl

def start_flower_server():
    # Define the Federated Averaging strategy
    strategy = fl.server.strategy.FedAvg()

    # Start the server with the strategy and configuration
    fl.server.start_server(
        server_address="localhost:8087",
        strategy=strategy,
        config={"num_rounds": 1},
    )

if __name__ == "__main__":
    # Start the server directly in the main thread
    start_flower_server()


	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower server, config: {'num_rounds': 1}


SystemExit: Port in server address localhost:8086 is already in use.

Once we change the server address from localhost:8086 to 8087, it runs smoothly. Every time it runs, the server address needs to be changed.

<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Run the Flower Client</b></div>

In [ ]:
from multiprocessing import Process
import flwr as fl

def start_flower_client(client_id):
    # Replace this with the actual implementation of your client
    print(f"Starting client {client_id}")
    model = create_model()
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = tf.keras.utils.to_categorical(y_train, 10), tf.keras.utils.to_categorical(y_test, 10)

    # Create and start the Flower client
    client = FlowerClient(model, x_train, y_train, x_test, y_test)
    fl.client.start_client(
        server_address="localhost:8080",  # Replace with your server address
        client=client.to_client(),
    )

if __name__ == "__main__":
    num_clients = 3  # Adjust the number of clients as needed

    # Start clients in separate processes
    client_processes = []
    for client_id in range(num_clients):
        process = Process(target=start_flower_client, args=(client_id,))
        client_processes.append(process)
        process.start()

    # Wait for all clients to finish
    for process in client_processes:
        process.join()


Starting client 0
Starting client 1
Starting client 2


2024-11-25 17:12:24.593824: F external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:164] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2024-11-25 17:12:24.596374: F external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:164] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2024-11-25 17:12:24.615518: F external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:164] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error


<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Evaluate the Final Global Model</b></div>

After federated training is complete, evaluate the final model on the test set:

In [ ]:
# Get the final weights from the trained global model
global_model_parameters = global_model.get_weights()

# Use the final weights to evaluate
final_model = create_model()
final_model.set_weights(global_model_parameters)
loss, accuracy = final_model.evaluate(x_test, y_test, verbose=0)
print(f"Final model accuracy: {accuracy:.4f}")


Final model accuracy: 0.7000


<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b> Final/More Evaluation</b></div>

In [ ]:
# After federated learning rounds
num_rounds = 5
for round_num in range(num_rounds):
    print(f"Round {round_num + 1}/{num_rounds}")

    local_weights = []

    # Train on each client
    for client_id in range(num_clients):
        print(f"Training on client {client_id + 1}")

        local_model = create_model()
        local_model.set_weights(global_model.get_weights())

        # Train local model
        X, y = client_data[client_id]
        local_model.fit(X, y, epochs=1, batch_size=32, verbose=0)

        # Collect local weights
        local_weights.append(local_model.get_weights())

    # Federated averaging
    averaged_weights = [np.mean([local_weights[j][i] for j in range(num_clients)], axis=0)
                        for i in range(len(local_weights[0]))]
    global_model.set_weights(averaged_weights)

    # Evaluate global model
    loss, accuracy = global_model.evaluate(x_test, y_test, verbose=0)
    print(f"Global model accuracy after round {round_num + 1}: {accuracy:.4f}")

# Save final weights
global_model_parameters = global_model.get_weights()

# Final evaluation
final_model = create_model()
final_model.set_weights(global_model_parameters)
loss, accuracy = final_model.evaluate(x_test, y_test, verbose=0)
print(f"Final model accuracy: {accuracy:.4f}")


Round 1/5
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 1: 0.7102
Round 2/5
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 2: 0.7100
Round 3/5
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 3: 0.7156
Round 4/5
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 4: 0.7188
Round 5/5
Training on client 1
Training on client 2
Training on client 3
Global model accuracy after round 5: 0.7161
Final model accuracy: 0.7161


<a id="1"></a>
# <div style="text-align:center; border-radius:25px 70px; padding:9px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#FF7F50; overflow:hidden"><b>Conclusion</b></div>